# Add tools
- Doc: https://langchain-ai.github.io/langgraph/tutorials/get-started/2-add-tools/

In [ ]:
from dotenv import load_dotenv

# API-KEY 읽어오기
load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model

# 모델 초기화
llm = init_chat_model("google_genai:gemini-2.5-flash")

## 3. Define the tool
- https://www.tavily.com/

In [ ]:
from langchain_tavily import TavilySearch

# 웹 검색을 도구
tool = TavilySearch(max_results=2)
# 도구 리스트
tools = [tool]
# 테스트
tool.invoke("What's a 'node' in LangGraph?")

In [ ]:
import json

# LLM에 입력 되는 값 확인
print("**** 도구 이름 ****")
print(tool.name)
print()
print("**** 도구 설명 ****")
print(tool.description)
print()
print("**** 도구 인자 ****")
print(json.dumps(tool.args_schema.model_json_schema(), indent=2))

## 4. Define the graph

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


# 에이전트의 상태 정의 클래스
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 상태 기반 워크플로우 생성
graph_builder = StateGraph(State)

# LLM이 도구 호출 여부 판단
llm_with_tools = llm.bind_tools(tools)

# chatbot 노드 함수
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 워크플로우에 chatbot 노드 추가
graph_builder.add_node("chatbot", chatbot)

## 5. Create a function to run the tools

In [ ]:
import json

from langchain_core.messages import ToolMessage


# 도구 실행 클래스
class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: State):
        # 입력 상태에서 메시지 리스트를 추출.
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        
        outputs = []
        for tool_call in message.tool_calls:
            # 도구 이름을 이용해 도구 호출
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            # 도구 결과를 `ToolMessage` 형식으로 변환하여 출력 리스트에 추가
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        # 도구 호출 결과를 포함한 메시지 리스트를 반환
        return {"messages": outputs}

# BasicToolNode 인스턴스 생성
tool_node = BasicToolNode(tools=[tool])
# tools 노드를 워크플로우에 추가
graph_builder.add_node("tools", tool_node)

## 6. Define the conditional_edges

In [ ]:
def route_tools(state: State):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]  # 마지막 메시지 (LLM 응답)
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    
    # LLM 응답에 `tool_calls`가 있으면 `tools` 노드로 이동.
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    # 없으면 END 노드로 이동 (종료)
    return END


# 조건부 라우팅: tools로 이동하거나 END로 종료.
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    # 라우팅 결과를 노드 이름으로 매핑
    {"tools": "tools", END: END},
)

# tools 노드 실행 후 chatbot 노드로 다시 이동 (도구 결과 처리)
graph_builder.add_edge("tools", "chatbot")
# 워크플로우 시작점에서 chatbot 노드로 이동
graph_builder.add_edge(START, "chatbot")
# 워크플로우 컴파일, 실행 가능한 워크플로우 생성
graph = graph_builder.compile()

## 7. Visualize the graph (optional)

In [ ]:
from IPython.display import Image, display

try:
    # 워크플로우 그래프를 이미지로 시각화하여 출력
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

## 8. Ask the bot questions

In [ ]:
def stream_graph_updates(user_input: str):
    # 워크플로우를 스트리밍 모드로 실행 (사용자 입력 처리)
    events = graph.stream({"messages": [{"role": "user", "content": user_input}]})
    for event in events:
        for value in event.values():
            # 이벤트의 마지막 메시지 출력
            value["messages"][-1].pretty_print()


while True:
    try:
        user_input = input("User: ")  # 사용자 입력 받기
        if user_input.lower() in ["quit", "exit", "q"]:  # 종료 조건
            print("Goodbye!")
            break

        # 워크플로우 실행
        stream_graph_updates(user_input)
    except:
        # 오류 발생시 기본 질문으로 대체
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

## 9. Use prebuilts

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# 에이전트의 상태 정의 클래스
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 상태 기반 워크플로우 생성
graph_builder = StateGraph(State)

# 웹 검색을 도구
tool = TavilySearch(max_results=2)
# 도구 리스트
tools = [tool]

# LLM이 도구 호출 여부 판단
llm_with_tools = llm.bind_tools(tools)

# chatbot 노드 함수
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 워크플로우에 chatbot 노드 추가
graph_builder.add_node("chatbot", chatbot)

# LangGraph의 prebuilt ToolNode를 사용
tool_node = ToolNode(tools=[tool])
# tool 노드 워크플로우에 추가
graph_builder.add_node("tools", tool_node)

# 조건부 라우팅: tools로 이동하거나 END로 이동 (종료)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,  # LangGraph의 prebuilt 함수
                      # 마지막 메시지에 `tool_calls`가 있는지 확인
)

# tools 노드 실행 후 chatbot 노드로 다시 이동 (도구 결과 처리)
graph_builder.add_edge("tools", "chatbot")
# 워크플로우 시작점에서 chatbot 노드로 이동
graph_builder.add_edge(START, "chatbot")
# 워크플로우 컴파일, 실행 가능한 워크플로우 생성
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    # 워크플로우 그래프를 이미지로 시각화하여 출력
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
def stream_graph_updates(user_input: str):
    # 워크플로우를 스트리밍 모드로 실행 (사용자 입력 처리)
    events = graph.stream({"messages": [{"role": "user", "content": user_input}]})
    for event in events:
        for value in event.values():
            # 이벤트의 마지막 메시지 출력
            value["messages"][-1].pretty_print()


while True:
    try:
        user_input = input("User: ")  # 사용자 입력 받기
        if user_input.lower() in ["quit", "exit", "q"]:  # 종료 조건
            print("Goodbye!")
            break

        # 워크플로우 실행
        stream_graph_updates(user_input)
    except:
        # 오류 발생시 기본 질문으로 대체
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break